# Winter 2022 Data Science Intern Challenge

## Question 1
>On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

**a. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

df = pd.read_csv("../input/Shopify 2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")

In [ ]:
df.order_amount.describe()

I'm interested in where that AOV of 3145.13 came from. I'm guessing it is probably from the mean as high values of order amounts will drag this number up. Using panda's handy `describe()` method on the data frame, we can see that the mean order amount is indeed 3,145.13. Also, there is an incredibly large standard deviation of 41282.54. This means that on average, the values vary 41,282.54 from the mean, making the mean not a very useful representation of the AOV!

The `describe()` function we used also gave us some valuable information to better understand this data aside from the mean and standard deviation.  We can see the minimum value is 90 and max value is 704,000. We can also see the values for the median and first and third quartile. Comparing these numbers, we can see that the maximum value is much higher than the other values. Considering our dataset has 5000 records, there are probably a few outlier values that are incredibly high dragging up the mean.

To get a sense of the distribution, we can look at a box plot of the data.

In [ ]:
df.boxplot(column='order_amount')

The entire box portion of the box plot is a line along 0! It looks like there are lots of outliers. Let's try finding them by grouping the data by order amount, counting the number of entries for each group, and sort it in descending order.

In [ ]:
unique_amounts = df.groupby(['order_amount']).size().reset_index(name='count').sort_values(by='order_amount', ascending=False)
unique_amounts.head(10)

Interesting. It looks like some of these high order amounts are repeated, especially 704000, 51450, and 25725. Let's look at these rows from our original dataset. 

In [ ]:
df.loc[df['order_amount'].isin([704000, 51450, 25725])].sort_values(by='order_amount', ascending=False)

Again this is interesting. It appears the order amounts of 704000 occur at the same time each day between the same store and users. The data for order amounts of 51450 and 25725 similar. It appears as though orders of 51450 are just transactions that bought two items worth 25725 as they all come from the same store id 78.

For the 704000 amounts, it seems like the transactions are probably some sort of supplier purchasing many shoes at once since the order amount is consistently 2000.

To better evaluate this data, we can clean some of the values. Let's try plotting only values that are the median +/- 1.5 times the interquartile range.

In [ ]:
q1 = df.order_amount.quantile(q=0.25)
q2 = df.order_amount.quantile(q=0.5)
q3 = df.order_amount.quantile(q=0.75)
IQR = q3 - q1

df_truncated = df[(df.order_amount < q2 + IQR * 1.5) & (df.order_amount > q2 - IQR * 1.5)]
df_truncated.boxplot(column='order_amount')

Using `df_truncated` should provide a more accurate representation of the typical order data.

**b. What metric would you report for this dataset? **

Looking at the box plot for `df_truncated`, it looks like the distribution is skewed towards lower values.  With this in mind, I would report the median value of the truncated dataset. The values above the median will increase the increase the mean a disproportionate amount.


**c. What is its value?**

We can find this value using the `describe()` function to find that it $272. It also shows that the standard deviation is 132.06, a much more reasonable result!

In [ ]:
df_truncated.order_amount.describe()

## Question 2
> For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

**a. How many orders were shipped by Speedy Express in total?**

If we join the `[Orders]` and `[Shippers]` tables on `ShipperID`, we can filter table by orders performed by `Speedy Express` and count the entries. 

```
SELECT COUNT(*) AS NumberOfOrders
FROM [Orders]
JOIN [Shippers]
	ON [Shippers].ShipperID = [Orders].ShipperID
WHERE [Shippers].ShipperName = 'Speedy Express'
```

Using the above query, the output shows the number of orders is **54**.

**b. What is the last name of the employee with the most orders?**

To solve this problem, we need to count the number of orders associated with each employee, see which one has the most orders, and report back their last name.

With SQL, we could count the number of orders in `[Orders]` and group by `EmployeeID`. This would give us which employee ID has the most orders, but not that employee's last name. Instead, we can use the `JOIN` expression to merge the `[Orders]` table and the `[Employees]` to match the employees to their employee ID and group by their last name to solve the problem.


```
SELECT [Employees].LastName, COUNT(*) AS NumberOfOrders
FROM [Orders]
JOIN [Employees]
ON [Orders].EmployeeID = [Employees].EmployeeID
GROUP BY [Employees].LastName
ORDER BY NumberOfOrders DESC
LIMIT 1
```

Using the query above shows that the employee with the last name **Peacock** had the most orders at **40**.

**c. What product was ordered the most by customers in Germany?**

The data we need to solve this problem is scattered across a few different tables. Our final query should show a list of products ordered by customers in Germay and how many orders of that product there were.

We can break down the problem into smaller sections and tackle them one at a time. First, let's look at *all orders from Germany*.

```
SELECT [Orders].OrderID,
	[Customers].Country
FROM [Orders]
JOIN [Customers]
	ON [Customers].CustomerID = [Orders].CustomerID
WHERE [Customers].Country = 'Germany'
```
This gives us a list of all orders to customers in Germany.

The next part of the problem is figuring out *which item was ordered the most*. We need to incorporate information such as product ID and quantity from the `[OrderDetails]` table to do this. By joining `[OrderDetails]` through the `OrderID` column, we can find the total quantity for each product by summing the quantity column and grouping by product id. 

```
SELECT [Customers].Country,
	[OrderDetails].ProductID,
    SUM([OrderDetails].Quantity) AS "TotalOrdered"
FROM [Orders]
JOIN [Customers]
	ON [Customers].CustomerID = [Orders].CustomerID
JOIN [OrderDetails]
	ON [OrderDetails].OrderID = [Orders].OrderID
WHERE [Customers].Country = 'Germany'
GROUP BY [OrderDetails].ProductID
ORDER BY TotalOrdered DESC -- Show most ordered item at the top.
```

At this point, we can see the most ordered item has a product ID of 40 at 160 orders. To make sense of this result, we can join the `[Products]` table on the `ProductID` column to get the name of the most ordered product.

```
SELECT [Products].ProductName,
    SUM([OrderDetails].Quantity) AS "TotalOrdered"
FROM [Orders]
JOIN [Customers]
	ON [Customers].CustomerID = [Orders].CustomerID
JOIN [OrderDetails]
	ON [OrderDetails].OrderID = [Orders].OrderID
JOIN [Products]
	ON [Products].ProductID = [OrderDetails].ProductID
WHERE [Customers].Country = 'Germany'
GROUP BY [OrderDetails].ProductID
ORDER BY TotalOrdered DESC -- Show most ordered item at the top.
```

This final query shows that **Boston Crab Meat** has the most orders at **160 total orders**.
